In [1]:
pwd

'c:\\Users\\LeeJiSu\\Desktop\\k디털플젝'

In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')


# 형태소 추출을 위한 라이브러리
import re

# 명사 형태소 추출을 위한 라이브러리
from konlpy.tag._okt import Okt
from collections import Counter

In [3]:
df = pd.read_csv('new.csv')

In [4]:
df.head()

,rating,review
0,10,인피니티 사가 최고의 마무리. 이보다 더 완벽할 순 없다.\n\n캡틴 아메리카에게 ...
1,10,아이언맨으로 시작된 10년 마블 영화 팬들에 대한 위대한 엔드 프리젠트.\r\n ...
2,10,"마블시네마틱유니버스의 대서사를 마무리 짓는 작품. 슈퍼 히어로와 액션이 난무하지만,..."
3,10,11년의 긴 세월을 사랑 받아온 한편의 드라마를 어떻게하면 화려하게 장식 할 수 있...
4,9,&#39;인피니티워&#39;에서 엄청난 전투씬의 융단폭격을 퍼붓던 기조가 &#39;...


In [5]:
# rating(점수) 와 review(후기)를의 2가지 열로 구성된 4310개의 행

df.shape

(4310, 2)

In [6]:
# 리뷰의 결측치 존재 확인

df.isnull().sum()

rating      0
review    861
dtype: int64

In [7]:
# 점수는 메겼지만 리뷰는 쓰지 않은 댓글 존재
# 4310 - 3449 = 861

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4310 entries, 0 to 4309
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   rating  4310 non-null   int64 
 1   review  3449 non-null   object
dtypes: int64(1), object(1)
memory usage: 67.5+ KB


In [8]:
# 아래의 두 리뷰를 보면 \n 이나 ㅋㅋ 등의 "특수 문자"와 "모음"이 존재하는 경우 다수
# 이들은 Text Mining을 적용할 의미가 없기에 정규표현식을 이용해 제거한다.

df['review'][0]

'인피니티 사가 최고의 마무리. 이보다 더 완벽할 순 없다.\n\n캡틴 아메리카에게 영향을 받아 변화한 하워드 스타크\n그런 과거의 아버지에게 영향을 받아 변화한 토니 스타크\n토니 스타크에게 영향을 받아 변화한 캡틴 아메리카\n\n결국 이기적이던 토니는 아이언맨으로써 모두를 위해 희생했고, 쉬지 않고 평생을 바쳐 활동하던 캡틴 아메리카는 은퇴하고 스티브 로져스의 삶을 산다..\n\n개인적으로 진짜 마블 영화 보면서 소름돋았던 것은 처음이다. 그것도 여러번이나.\n- 아이언맨의 핑거 스냅\n- 토니 스타크의 "3000만큼 사랑해"\n- 캡틴 아메리카의 "어벤져스 어셈블"\n- 스티븐 로저스와 페기 카터의 댄스\n\n아이언맨은 1편에서 했던 명대사로 끝을 맞았고, 스티븐은 1편에서 했던 약속을 지키며 끝을 맞았다'

In [9]:
df['review'][120]

'전편들이나 마블 시리즈 스토리, 캐릭터를 대강 아는 사람들은 꿀잼, 내용 모르고 처음 보는 사람들은 당연히 지루하지ㅋㅋ'

In [20]:
# 한국어 텍스트 데이터 전처리
# 리뷰의 평가 내용을 단어화해서 형태소를 추출하고,
# 그 다음 Bag of Words를 생성하여 TF-IDF 변환을 진행하겠습니다.
# 한글 형태소 분석기인 KoNLPy를 설치하는 과정

!pip install konlpy==0.5.2 jpype1 Jpype1-py3

Defaulting to user installation because normal site-packages is not writeable
  Using cached konlpy-0.5.2-py2.py3-none-any.whl (19.4 MB)
  Attempting uninstall: konlpy
    Found existing installation: konlpy 0.6.0
    Uninstalling konlpy-0.6.0:
      Successfully uninstalled konlpy-0.6.0


In [11]:
# pip install --upgrade konlpy

In [12]:
# pip install --upgrade jpype1

In [13]:
# 정규 표현식 함수 정의

def apply_regular_expression(text):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 한글 추출 규칙: 띄어 쓰기(1 개)를 포함한 한글
    result = hangul.sub('', text)  # 위에 설정한 "hangul"규칙을 "text"에 적용(.sub)시킴
    return result

In [14]:
# 정규 표현식을 통해 "특수 문자"와 "모음" 제거

apply_regular_expression(df['review'][0])

'인피니티 사가 최고의 마무리 이보다 더 완벽할 순 없다캡틴 아메리카에게 영향을 받아 변화한 하워드 스타크그런 과거의 아버지에게 영향을 받아 변화한 토니 스타크토니 스타크에게 영향을 받아 변화한 캡틴 아메리카결국 이기적이던 토니는 아이언맨으로써 모두를 위해 희생했고 쉬지 않고 평생을 바쳐 활동하던 캡틴 아메리카는 은퇴하고 스티브 로져스의 삶을 산다개인적으로 진짜 마블 영화 보면서 소름돋았던 것은 처음이다 그것도 여러번이나 아이언맨의 핑거 스냅 토니 스타크의 만큼 사랑해 캡틴 아메리카의 어벤져스 어셈블 스티븐 로저스와 페기 카터의 댄스아이언맨은 편에서 했던 명대사로 끝을 맞았고 스티븐은 편에서 했던 약속을 지키며 끝을 맞았다'

In [15]:
df['review'][0]

'인피니티 사가 최고의 마무리. 이보다 더 완벽할 순 없다.\n\n캡틴 아메리카에게 영향을 받아 변화한 하워드 스타크\n그런 과거의 아버지에게 영향을 받아 변화한 토니 스타크\n토니 스타크에게 영향을 받아 변화한 캡틴 아메리카\n\n결국 이기적이던 토니는 아이언맨으로써 모두를 위해 희생했고, 쉬지 않고 평생을 바쳐 활동하던 캡틴 아메리카는 은퇴하고 스티브 로져스의 삶을 산다..\n\n개인적으로 진짜 마블 영화 보면서 소름돋았던 것은 처음이다. 그것도 여러번이나.\n- 아이언맨의 핑거 스냅\n- 토니 스타크의 "3000만큼 사랑해"\n- 캡틴 아메리카의 "어벤져스 어셈블"\n- 스티븐 로저스와 페기 카터의 댄스\n\n아이언맨은 1편에서 했던 명대사로 끝을 맞았고, 스티븐은 1편에서 했던 약속을 지키며 끝을 맞았다'

In [23]:
okt = Okt()
nouns = okt.nouns(apply_regular_expression(df['review'][5]))
nouns

TypeError: startJVM() got an unexpected keyword argument 'classpath'